Configuring the model


In [61]:
from dotenv import load_dotenv
import os
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY") # here i configured my api key from .env file

In [62]:
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model='gemini-1.5-flash')
output = model.invoke("hi hello")
print(output.content)



Hi there! How can I help you today?


Configuring the embedding model 

In [63]:
from dotenv import load_dotenv
import os
load_dotenv()
os.environ["HUGGINGFACE_API_KEY"] = os.getenv("HUGGINGFACE_API_KEY") # here i configured my api key from .env file



In [64]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en")
len(embeddings.embed_query("hi"))

384

LETS TAKE A DATA EMBEDED IT AND STORE IT IN VECTOR DATABASE

In [65]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [66]:
loader=DirectoryLoader("../data",glob="./*.txt",loader_cls=TextLoader) #here is the directory name data from parent directory

In [67]:
docs=loader.load()

In [68]:
docs

[Document(metadata={'source': '../data/Indiaeconomy.txt'}, page_content="Certainly! Here's a well-organized and detailed write-up on **India's economy in 2025** presented in paragraph format:\n\n---\n\n**India’s Economic Landscape in 2025**\n\nIndia stands tall in 2025 as one of the most dynamic and rapidly expanding economies in the world. With a nominal GDP of approximately **USD 4.19 trillion**, India is currently ranked **fourth globally**, just behind the U.S., China, and Germany. When adjusted for purchasing power parity (PPP), India's GDP climbs to **USD 17.65 trillion**, making it the **third-largest economy** worldwide. This robust economic performance is driven by structural reforms, technological advancement, and a favorable demographic dividend.\n\nThe **real GDP growth rate** for the fiscal year 2024–25 is estimated at **6.4%**, slightly lower than the previous year’s impressive 8.2% but still significantly higher than most other major economies. This pace confirms India’s

In [69]:
docs[0].page_content

"Certainly! Here's a well-organized and detailed write-up on **India's economy in 2025** presented in paragraph format:\n\n---\n\n**India’s Economic Landscape in 2025**\n\nIndia stands tall in 2025 as one of the most dynamic and rapidly expanding economies in the world. With a nominal GDP of approximately **USD 4.19 trillion**, India is currently ranked **fourth globally**, just behind the U.S., China, and Germany. When adjusted for purchasing power parity (PPP), India's GDP climbs to **USD 17.65 trillion**, making it the **third-largest economy** worldwide. This robust economic performance is driven by structural reforms, technological advancement, and a favorable demographic dividend.\n\nThe **real GDP growth rate** for the fiscal year 2024–25 is estimated at **6.4%**, slightly lower than the previous year’s impressive 8.2% but still significantly higher than most other major economies. This pace confirms India’s status as the **fastest-growing major economy** globally. Key contribut

In [70]:
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 50,
)

In [71]:
new_docs=text_splitter.split_documents(documents=docs)

In [72]:
new_docs

[Document(metadata={'source': '../data/Indiaeconomy.txt'}, page_content="Certainly! Here's a well-organized and detailed write-up on **India's economy in 2025** presented in paragraph format:\n\n---\n\n**India’s Economic Landscape in 2025**"),
 Document(metadata={'source': '../data/Indiaeconomy.txt'}, page_content='India stands tall in 2025 as one of the most dynamic and rapidly expanding economies in the world. With a nominal GDP of approximately **USD 4.19 trillion**, India is currently ranked **fourth'),
 Document(metadata={'source': '../data/Indiaeconomy.txt'}, page_content="trillion**, India is currently ranked **fourth globally**, just behind the U.S., China, and Germany. When adjusted for purchasing power parity (PPP), India's GDP climbs to **USD 17.65 trillion**,"),
 Document(metadata={'source': '../data/Indiaeconomy.txt'}, page_content="India's GDP climbs to **USD 17.65 trillion**, making it the **third-largest economy** worldwide. This robust economic performance is driven by

In [73]:
doc_string=[doc.page_content for doc in new_docs]

In [74]:
doc_string


["Certainly! Here's a well-organized and detailed write-up on **India's economy in 2025** presented in paragraph format:\n\n---\n\n**India’s Economic Landscape in 2025**",
 'India stands tall in 2025 as one of the most dynamic and rapidly expanding economies in the world. With a nominal GDP of approximately **USD 4.19 trillion**, India is currently ranked **fourth',
 "trillion**, India is currently ranked **fourth globally**, just behind the U.S., China, and Germany. When adjusted for purchasing power parity (PPP), India's GDP climbs to **USD 17.65 trillion**,",
 "India's GDP climbs to **USD 17.65 trillion**, making it the **third-largest economy** worldwide. This robust economic performance is driven by structural reforms, technological advancement, and a",
 'reforms, technological advancement, and a favorable demographic dividend.',
 'The **real GDP growth rate** for the fiscal year 2024–25 is estimated at **6.4%**, slightly lower than the previous year’s impressive 8.2% but still si

In [75]:
len(doc_string)

31

In [78]:
db=Chroma.from_documents(documents=new_docs, embedding=embeddings)

In [80]:
retriver=db.as_retriever(search_kwargs={"k":3})

In [81]:
retriver.invoke("growth of indian economy in 2025")

[Document(metadata={'source': '../data/Indiaeconomy.txt'}, page_content='India stands tall in 2025 as one of the most dynamic and rapidly expanding economies in the world. With a nominal GDP of approximately **USD 4.19 trillion**, India is currently ranked **fourth'),
 Document(metadata={'source': '../data/Indiaeconomy.txt'}, page_content='India stands tall in 2025 as one of the most dynamic and rapidly expanding economies in the world. With a nominal GDP of approximately **USD 4.19 trillion**, India is currently ranked **fourth'),
 Document(metadata={'source': '../data/Indiaeconomy.txt'}, page_content='India stands tall in 2025 as one of the most dynamic and rapidly expanding economies in the world. With a nominal GDP of approximately **USD 4.19 trillion**, India is currently ranked **fourth')]

USING PYDANTIC FOR CLASS CREATION

In [107]:

import operator
from typing import List
from pydantic import BaseModel , Field
from langchain.prompts import PromptTemplate
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph,END

In [108]:
from pydantic import BaseModel, Field

In [109]:
class TopSelectionParser(BaseModel):
    Topic:str=Field(description="Select a topic")
    Reasoning:str=Field(description="reasoning behind Selecting a topic")

In [110]:
from langchain.output_parsers import PydanticOutputParser

In [111]:
parser =PydanticOutputParser(pydantic_object=TopSelectionParser)

In [112]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"Topic": {"description": "Select a topic", "title": "Topic", "type": "string"}, "Reasoning": {"description": "reasoning behind Selecting a topic", "title": "Reasoning", "type": "string"}}, "required": ["Topic", "Reasoning"]}\n```'

In [115]:

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]